영양 상태를 기반으로 영양제 추천 -> 식단 추천으로 얼마든지 변경이 가능하다는 생각

Cosine similarity 최대화 방향성 -> 이친구가 더 좋다는듯?
SSE(편차 제곱) 최소화 방향성 -> ㄴㄴ

어떤 식으로 다가가야 할 지 고민해야 합니다.

-> 예시를 들어서
        
        표준화 모델이니까 반반떄리다 치고

        1. 남은 영양성분

        2. 먹은 아점저 정보 (식단관리창에서)

        3. 아/점/저 -> 33% 해가지고 이렇게 하자? ㅇㅋ 

        4. 건강한 식단을 위해서는 아침>점심>저녁 인데 이걸 어캐 표현을 하나?

        5. 머신러닝 뭘로 할지? 
        - Cosine similarity인데....        

기본 식 -> 일일 활동량 비교해서 만듬 -> DB에서 해줄꺼고
음식영양성분 데이터 있으니까 여기서 URL을 담아둔 친구들을 사용해야 합니다.


무엇을 받아올까요?

- 웹에서 받아올 수 있는 부분
1. 무엇을 먹었는지 -> 남은 탄,단,지,열량
2. 아침,점심,저녁을 먹었는지 안먹었는지 -=> 이건 해야함.

In [1]:
import os
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import json
from tqdm import tqdm
import re
import sys

In [2]:
%pwd

'C:\\Users\\smhrd\\Yolov7_Detect'

In [3]:
data = pd.read_excel(r'C:/Users/smhrd/food_info_.xlsx')
data

,FOOD_SEQ,음식명,레시피,한중일양,밥국반메면,식품군,제조사명,지역명,중량,열량(kcal),탄수화물(g),단백질(g),지방(g),당류(g),나트륨(mg),콜레스트롤(mg),포화지방산(g),트랜스지방(g)
0,1,꿩불고기,https://www.youtube.com/embed/rl5xaIdHjPY,북,메,NaN,NaN,충주,500.0,368.80,39.70,33.50,8.50,16.90,1264.31,106.18,1.90,0.10
1,2,닭갈비,https://www.youtube.com/embed/HI8DlDs1gko,한,메,NaN,NaN,전국(대표),400.0,595.61,44.90,45.90,25.80,21.20,1535.83,193.40,6.00,0.20
2,3,닭갈비,https://www.youtube.com/embed/HI8DlDs1gko,한,메,NaN,NaN,춘천,300.0,558.47,23.10,45.50,31.60,8.50,1016.94,217.67,11.30,0.30
3,4,닭꼬치,https://www.youtube.com/embed/1xrcjy6eX7M,한,메,NaN,NaN,전국(대표),70.0,176.72,13.35,11.56,8.57,3.15,286.91,31.44,2.32,0.04
4,5,더덕구이,https://www.youtube.com/embed/ir4Z2IyFIvM,한,반,NaN,NaN,전국(대표),100.0,184.00,31.10,3.10,5.20,11.60,743.37,0.12,0.80,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90603,90604,"코코아, 가루",NaN,NaN,NaN,차류,NaN,전국(대표),100.0,268.00,43.63,22.85,20.06,0.49,24.00,0.00,12.11,0.01
90604,90605,"홍삼차, 액상",NaN,NaN,NaN,차류,NaN,전국(대표),100.0,40.00,9.27,0.28,0.24,6.32,4.00,0.00,0.00,0.00
90605,90606,"홍차, 복숭아, 가루",NaN,NaN,NaN,차류,NaN,전국(대표),100.0,397.00,98.12,0.32,0.38,42.31,41.00,0.00,0.00,0.00
90606,90607,"마요네즈, 저칼로리",NaN,NaN,NaN,조미료류,NaN,전국(대표),100.0,341.00,12.35,0.89,31.97,3.56,600.00,23.72,3.96,0.22


In [4]:
data.head(3)

,FOOD_SEQ,음식명,레시피,한중일양,밥국반메면,식품군,제조사명,지역명,중량,열량(kcal),탄수화물(g),단백질(g),지방(g),당류(g),나트륨(mg),콜레스트롤(mg),포화지방산(g),트랜스지방(g)
0,1,꿩불고기,https://www.youtube.com/embed/rl5xaIdHjPY,북,메,NaN,NaN,충주,500.0,368.80,39.7,33.5,8.5,16.9,1264.31,106.18,1.9,0.1
1,2,닭갈비,https://www.youtube.com/embed/HI8DlDs1gko,한,메,NaN,NaN,전국(대표),400.0,595.61,44.9,45.9,25.8,21.2,1535.83,193.40,6.0,0.2
2,3,닭갈비,https://www.youtube.com/embed/HI8DlDs1gko,한,메,NaN,NaN,춘천,300.0,558.47,23.1,45.5,31.6,8.5,1016.94,217.67,11.3,0.3


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90608 entries, 0 to 90607
Data columns (total 18 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   FOOD_SEQ   90608 non-null  int64  
 1   음식명        90608 non-null  object 
 2   레시피        829 non-null    object 
 3   한중일양       829 non-null    object 
 4   밥국반메면      755 non-null    object 
 5   식품군        82904 non-null  object 
 6   제조사명       85289 non-null  object 
 7   지역명        90608 non-null  object 
 8   중량         90608 non-null  float64
 9   열량(kcal)   90608 non-null  float64
 10  탄수화물(g)    90608 non-null  float64
 11  단백질(g)     90608 non-null  float64
 12  지방(g)      90608 non-null  float64
 13  당류(g)      90608 non-null  float64
 14  나트륨(mg)    90608 non-null  float64
 15  콜레스트롤(mg)  90608 non-null  float64
 16  포화지방산(g)   90608 non-null  float64
 17  트랜스지방(g)   90608 non-null  float64
dtypes: float64(10), int64(1), object(7)
memory usage: 12.4+ MB


In [18]:
# 필요없는 부분 -> Food_group, Food_mf 등등 삭제
# 쓸 부분 - seq, name, recipe, weigth, calor, crb, protein, fat 씁니다.
# 컬럼 제거
data.drop(
    ['식품군','제조사명','지역명','당류(g)','나트륨(mg)','콜레스트롤(mg)'
    ,'포화지방산(g)','트랜스지방(g)'],
    axis=1,
    errors='ignore',
    inplace=True
)

In [19]:
data.head(3)

,FOOD_SEQ,음식명,레시피,한중일양,밥국반메면,중량,열량(kcal),탄수화물(g),단백질(g),지방(g)
0,1,꿩불고기,https://www.youtube.com/embed/rl5xaIdHjPY,북,메,500.0,368.80,39.7,33.5,8.5
1,2,닭갈비,https://www.youtube.com/embed/HI8DlDs1gko,한,메,400.0,595.61,44.9,45.9,25.8
2,3,닭갈비,https://www.youtube.com/embed/HI8DlDs1gko,한,메,300.0,558.47,23.1,45.5,31.6


In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 829 entries, 0 to 828
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   FOOD_SEQ  829 non-null    int64  
 1   음식명       829 non-null    object 
 2   레시피       829 non-null    object 
 3   한중일양      829 non-null    object 
 4   밥국반메면     755 non-null    object 
 5   중량        829 non-null    float64
 6   열량(kcal)  829 non-null    float64
 7   탄수화물(g)   829 non-null    float64
 8   단백질(g)    829 non-null    float64
 9   지방(g)     829 non-null    float64
dtypes: float64(5), int64(1), object(4)
memory usage: 64.9+ KB


In [21]:
# Food_name 전처리하기
data['음식명'] = data['음식명'].apply(lambda x: re.sub('\s(?![a-z])|_|\((.*?)\)|(.*?)\?','', str(x)))
data.head()

,FOOD_SEQ,음식명,레시피,한중일양,밥국반메면,중량,열량(kcal),탄수화물(g),단백질(g),지방(g)
0,1,꿩불고기,https://www.youtube.com/embed/rl5xaIdHjPY,북,메,500.0,368.80,39.70,33.50,8.50
1,2,닭갈비,https://www.youtube.com/embed/HI8DlDs1gko,한,메,400.0,595.61,44.90,45.90,25.80
2,3,닭갈비,https://www.youtube.com/embed/HI8DlDs1gko,한,메,300.0,558.47,23.10,45.50,31.60
3,4,닭꼬치,https://www.youtube.com/embed/1xrcjy6eX7M,한,메,70.0,176.72,13.35,11.56,8.57
4,5,더덕구이,https://www.youtube.com/embed/ir4Z2IyFIvM,한,반,100.0,184.00,31.10,3.10,5.20


In [22]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 829 entries, 0 to 828
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   FOOD_SEQ  829 non-null    int64  
 1   음식명       829 non-null    object 
 2   레시피       829 non-null    object 
 3   한중일양      829 non-null    object 
 4   밥국반메면     755 non-null    object 
 5   중량        829 non-null    float64
 6   열량(kcal)  829 non-null    float64
 7   탄수화물(g)   829 non-null    float64
 8   단백질(g)    829 non-null    float64
 9   지방(g)     829 non-null    float64
dtypes: float64(5), int64(1), object(4)
memory usage: 64.9+ KB


In [23]:
# 혹시 모르니까 수틀리면 지우기
# cols = ['food_name']
# drop_index = food_p_p.loc[:,cols][food_p_p.loc[:,cols].duplicated()].index
# food_p_p.drop(drop_index, axis=0, inplace=True)

In [24]:
# 중복 확인
data.iloc[:,:7][data.duplicated(subset=['FOOD_SEQ','음식명'])]
# 이러면 중복이 없다는 소리네요~

,FOOD_SEQ,음식명,레시피,한중일양,밥국반메면,중량,열량(kcal)


In [25]:
data['레시피'].isna()

0      False
1      False
2      False
3      False
4      False
       ...  
824    False
825    False
826    False
827    False
828    False
Name: 레시피, Length: 829, dtype: bool

In [26]:
# DF내 모든 값을 비교해 비어있으면 삭제
# data.dropna(axis=0, how='any',inplace=True)

# 레시피가 nan값이 아닌 애들로만 구성.
data = data[data['레시피'].notna()]
# 인덱스 값 새롭게 지정하기
data = data.reset_index(drop=True)
data

,FOOD_SEQ,음식명,레시피,한중일양,밥국반메면,중량,열량(kcal),탄수화물(g),단백질(g),지방(g)
0,1,꿩불고기,https://www.youtube.com/embed/rl5xaIdHjPY,북,메,500.0,368.80,39.70,33.50,8.50
1,2,닭갈비,https://www.youtube.com/embed/HI8DlDs1gko,한,메,400.0,595.61,44.90,45.90,25.80
2,3,닭갈비,https://www.youtube.com/embed/HI8DlDs1gko,한,메,300.0,558.47,23.10,45.50,31.60
3,4,닭꼬치,https://www.youtube.com/embed/1xrcjy6eX7M,한,메,70.0,176.72,13.35,11.56,8.57
4,5,더덕구이,https://www.youtube.com/embed/ir4Z2IyFIvM,한,반,100.0,184.00,31.10,3.10,5.20
...,...,...,...,...,...,...,...,...,...,...
824,7324,오징어튀김,https://www.youtube.com/embed/K7KCOvEKPbg,한,메,150.0,505.71,33.65,18.56,32.99
825,7325,채소튀김,https://www.youtube.com/embed/1uvcqAUjH8A,한,메,100.0,404.63,36.83,3.25,27.14
826,7326,치킨너겟,https://www.youtube.com/embed/aw1YhCxSF7c,한,메,150.0,468.40,14.93,22.47,35.42
827,7327,탕수육,https://www.youtube.com/embed/mzZNNMM2WuQ,중,NaN,200.0,358.97,24.84,27.21,16.75


미플린-지어(Mifflin-St.Jeor) 공식:

기본식: (10 × 몸무게) + (6.25 × 키) – (5 × 나이)

남성: 기본식 ＋ 5
여성: 기본식 － 161

기초대사량 ＋ 활동대사량 계산 식:

거의 운동하지 않음: 기초대사량 × 1.2
가벼운 운동(주 1~3일): 기초대사량 × 1.375
보통 수준: 기초대사량 × 1.55
적극적으로 운동(주 6~7일): 기초대사량 × 1.725
매우 적극적으로 운동(주 6~7일): 기초대사량 × 1.9

탄수화물: 1g당 4kcal
단백질: 1g당 4kcal
지방: 1g당 9kcal

SQL 구문을 만들어보아요
insert into member_info values (
  idman,
  12345,
  bususer,
  28,
  man,
  174.6,
  69.4,
  유지,
  1~3,


)

In [15]:
# 174 69 남성 유지 1.375 28

In [16]:
cal = ((10*69)+(6.25*174)-(5*28) +5)*1.375
cal

2258.4375

In [17]:
# cal을 55:25:20 으로 나눠야합니다.
crb = cal*11/20
protein = cal/5
fat =  cal/4
print(crb, protein, fat)

1242.140625 451.6875 564.609375


In [18]:
my_crb = crb/4
my_protein = protein/4
my_fat = fat/9
print(my_crb,my_protein,my_fat)

310.53515625 112.921875 62.734375


In [19]:
# 남성의 경우: BMR = 88.362 + (13.397 x 체중(kg)) + (4.799 x 키(cm)) - (5.677 x 나이(세))
new_cal = (88.362 + (13.397*69)+(4.799*174) - (5.677*28))
new_cal

1688.825

In [20]:
# 탄수화물: 1800kcal의 50-60%인 약 225-270g
# 단백질: 1800kcal의 15-20%인 약 68-90g
# 지방: 1800kcal의 20-30%인 약 40-60g

In [21]:
new_cal=new_cal*1.375
new_cal

2322.134375

In [22]:
new_crb = new_cal*0.5
new_protein = new_cal*0.15
new_fat = new_cal*0.35

In [23]:
new_crb = new_crb/4
new_protein = new_protein/4
new_fat = new_fat/9

In [24]:
print(new_crb,new_protein,new_fat)

290.266796875 87.0800390625 90.30522569444445


In [27]:
# 개인 정보에 기반한 활동 대사량, 기초대사량 구하기
# Physical Activity Level (PAL)
def user_PAL(weight, height, age, gender, activity, purpose):
  # 기초대사량 -> 키, 몸무게, 나이로 구성된 기본식
  base = (10*weight)+(6.25*height)-(5*age)
  # 남성일 경우
  if gender == "man":
    man_base=(base+5)
    # Eat-Fit을 쓰는 목적 -> 증량, 유지, 다이어트
    if purpose == "plus":
        man_base = man_base*1.2
        # 활동량 -> 1 : 아예 운동 X / 2 : 주 1~3 가벼운 운동 / 3 : 보통 수준 운동 / 4 : 적극적 운동 / 5: 매우 적극적 운동
        if activity == 1:
            man_base = man_base*1.2
        elif activity == 2:
            man_base = man_base*1.375
        elif activity == 3:
            man_base = man_base*1.55
        elif activity == 4:
            man_base = man_base*1.725
        elif activity == 5:
            man_base = man_base*1.9
    elif purpose == "maintain":
        man_base = man_base
        if activity == 1:
            man_base = man_base*1.2
        elif activity == 2:
            man_base = man_base*1.375
        elif activity == 3:
            man_base = man_base*1.55
        elif activity == 4:
            man_base = man_base*1.725
        elif activity == 5:
            man_base = man_base*1.9
    else :
        man_base = man_base*0.8
        if activity == 1:
            man_base = man_base*1.2
        elif activity == 2:
            man_base = man_base*1.375
        elif activity == 3:
            man_base = man_base*1.55
        elif activity == 4:
            man_base = man_base*1.725
        elif activity == 5:
            man_base = man_base*1.9

    crb=man_base*0.5
    protein = man_base*0.15
    fat = man_base*0.35

    man_crb = crb/4
    man_protein = protein/4
    man_fat = fat/9

    user = {}
    user = {'cal':man_base, 'crb':man_crb, 'protein':man_protein, 'fat':man_fat}
    return user

  else :
    woman_base = (base-161)
    if purpose == "plus":
        woman_base = woman_base*1.2
        if activity == 1:
            woman_base = woman_base*1.2
        elif activity == 2:
            woman_base = woman_base*1.375
        elif activity == 3:
            woman_base = woman_base*1.55
        elif activity == 4:
            woman_base = woman_base*1.725
        elif activity == 5:
            woman_base = woman_base*1.9
    elif purpose == "maintain":
        woman_base = woman_base
        if activity == 1:
            woman_base = woman_base*1.2
        elif activity == 2:
            woman_base = woman_base*1.375
        elif activity == 3:
            woman_base = woman_base*1.55
        elif activity == 4:
            woman_base = woman_base*1.725
        elif activity == 5:
            woman_base = woman_base*1.9
    else :
        woman_base = woman_base*0.8
        if activity == 1:
            woman_base = woman_base*1.2
        elif activity == 2:
            woman_base = woman_base*1.375
        elif activity == 3:
            woman_base = woman_base*1.55
        elif activity == 4:
            woman_base = woman_base*1.725
        elif activity == 5:
            woman_base = woman_base*1.9

        crb2=woman_base*0.5
        protein2 = woman_base*0.15
        fat2 = woman_base*0.35

        woman_crb = crb2/4
        woman_protein = protein2/4
        woman_fat = fat2/9

        user = {}
        user = {'cal':woman_base, 'crb':woman_crb, 'protein':woman_protein, 'fat':woman_fat}

    return user
  

In [28]:
#def user_PAL(weight, height, age, gender, activity, purpose):
user=user_PAL(72,176,25,'man',3,'maintain')
user

{'cal': 2635.0, 'crb': 329.375, 'protein': 98.8125, 'fat': 102.47222222222221}

In [29]:
user_cal=user['cal']
user_cal

2635.0

In [39]:
data2=data
data2

'FOOD_SEQ'

In [29]:
food_cal=data.loc[2, '열량(kcal)']
food_cal

558.47

In [30]:
user_cal = round(float(user_cal - food_cal),2)
user_cal

2076.53

In [31]:
# 내가 가진 영양정보에서 오늘 먹은 식단 제외하고 남은 영양 성분 구하는 공식
# 여기서 x는 먹은 음식의 food_seq입니다.
def remaining_nut(dic, df, x):
  # 여기서 dic은 상단의 user_PAL을 통해 만들어진 딕셔너리를 의미합니다
  # df 는 상단에서 변형한 data를 의미합니다!
  # df와 x를 넣는다면 후에 아래의 식처럼 보여줄 수 있습니다.

    user_cal=dic['cal']
    user_crb = dic['crb']
    user_protein = dic['protein']
    user_fat = dic['fat']

    food_cal = df.loc[x,'열량(kcal)']
    food_crb = df.loc[x,'탄수화물(g)']
    food_protein = df.loc[x,'단백질(g)']
    food_fat = df.loc[x,'지방(g)']

    remain_cal = round(float(user_cal-food_cal),3)
    remain_crb = round(float(user_crb-food_crb),3)
    remain_protein = round(float(user_protein-food_protein),3)
    remain_fat = round(float(user_fat-food_fat),3)

    user_dic = {}
    user_dic = {'cal':remain_cal, 'crb':remain_crb, 'protein':remain_protein, 'fat':remain_fat}

    return user_dic

In [32]:
# 식단을 먹고 남은 것을 기반으로 새로운 친구를 추출할 수 있지 않는가...
remain_user_data = remaining_nut(user, data, 1)
remain_user_data

{'cal': 2039.39, 'crb': 284.475, 'protein': 52.913, 'fat': 76.672}

In [33]:
remain_user_data['cal']

2039.39

In [34]:
# 현재 먹은 코드 => M,L,D
code = 'M'
# 이제 먹어야 할 코드 => M, L, D
now_code = 'L'

In [38]:
# # 나의 생각인데 식단을 추천할 것이라면??
# # 함수를 만들지만 아침 점심 저녁을 확인해야 함.
# # 아 점 저 로 확인한다면? (얘가 코드임)
# # code = > 리스트나 딕셔너리 형태여야 합니다. why? 아,점을 먹엇다면? 이런 경우가 잇다
# def recommend_dish(dic, df, code, now_code):
#   # 여기서 dic은 사용자의 정보
#     cal = dic['cal']
#     crb = dic['crb']
#     protein = dic['protein']
#     fat = dic['fat']
#   # df는 데이터베이스에 있는 친구 (영양정보데이터)
#     food_cal = df.loc[x,'열량(kcal)']
#     food_crb = df.loc[x,'탄수화물(g)']
#     food_protein = df.loc[x,'단백질(g)']
#     food_fat = df.loc[x,'지방(g)']

#   # 아점저 코드를 받는다면? 
#   # 아 : M  점 : L  저 : D
#   # flask로 받는데 이름을 정해야 함. ==> 여기선 code로 갑시다.
#   if 'M' in code and 'L' not in code and 'D' not in code:
#     # 아침만 먹은 상황 점심을 먹을 상황
#     cal = cal/3
#     crb = crb/3
#     protein = protein/3
#     fat = fat/3
#     if now_code == 'L':
#       # 점심에 추천할 음식 골라봅시다.
      
#     return
#   elif 'M' in code and 'L' in code and 'D' not in code:
#     # 아침과 점심을 먹은 상황 저녁 먹을상황

#     return
#   elif 'M' not in code and 'L' not in code and 'D' not in code:
#     # 아무것도 안먹었을 경우 => 아침을 먹어야 할 상황

#     return
#   elif 'M' not in code and 'L' in code and 'D' not in code:
#     # 아침X 점심만 먹은 경우 저녁을 먹을 상황

#     return
#   else : 
#     # 전혀 다른 경우

#     return
    

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 20)

1. 사용자의 남은 영양성분을 가지고 옵니다.
2. 사용자가 M을 먹었는지 L인지 D인지 가지고 옵니다.
3. 남은 영양성분, 이런건 일단 가데이터 놓고 나머지 로직 처리

In [13]:
#def user_PAL(weight, height, age, gender, activity, purpose):
user=user_PAL(69,174,28,'man',3,'maintain')
user

{'cal': 2545.875,
 'crb': 318.234375,
 'protein': 95.47031249999999,
 'fat': 99.00625}

In [14]:
user_cal=user['cal']
user_cal

2545.875

In [ ]:
# 현재 먹은 코드 => M,L,D
code = 'M'
# 이제 먹어야 할 코드 => M, L, D
now_code = 'L'

In [31]:
data

,FOOD_SEQ,음식명,레시피,한중일양,밥국반메면,중량,열량(kcal),탄수화물(g),단백질(g),지방(g)
0,1,꿩불고기,https://www.youtube.com/embed/rl5xaIdHjPY,북,메,500.0,368.80,39.70,33.50,8.50
1,2,닭갈비,https://www.youtube.com/embed/HI8DlDs1gko,한,메,400.0,595.61,44.90,45.90,25.80
2,3,닭갈비,https://www.youtube.com/embed/HI8DlDs1gko,한,메,300.0,558.47,23.10,45.50,31.60
3,4,닭꼬치,https://www.youtube.com/embed/1xrcjy6eX7M,한,메,70.0,176.72,13.35,11.56,8.57
4,5,더덕구이,https://www.youtube.com/embed/ir4Z2IyFIvM,한,반,100.0,184.00,31.10,3.10,5.20
...,...,...,...,...,...,...,...,...,...,...
824,7324,오징어튀김,https://www.youtube.com/embed/K7KCOvEKPbg,한,메,150.0,505.71,33.65,18.56,32.99
825,7325,채소튀김,https://www.youtube.com/embed/1uvcqAUjH8A,한,메,100.0,404.63,36.83,3.25,27.14
826,7326,치킨너겟,https://www.youtube.com/embed/aw1YhCxSF7c,한,메,150.0,468.40,14.93,22.47,35.42
827,7327,탕수육,https://www.youtube.com/embed/mzZNNMM2WuQ,중,NaN,200.0,358.97,24.84,27.21,16.75


In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 829 entries, 0 to 828
Data columns (total 18 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   FOOD_SEQ   829 non-null    int64  
 1   음식명        829 non-null    object 
 2   레시피        829 non-null    object 
 3   한중일양       829 non-null    object 
 4   밥국반메면      755 non-null    object 
 5   식품군        0 non-null      object 
 6   제조사명       0 non-null      object 
 7   지역명        829 non-null    object 
 8   중량         829 non-null    float64
 9   열량(kcal)   829 non-null    float64
 10  탄수화물(g)    829 non-null    float64
 11  단백질(g)     829 non-null    float64
 12  지방(g)      829 non-null    float64
 13  당류(g)      829 non-null    float64
 14  나트륨(mg)    829 non-null    float64
 15  콜레스트롤(mg)  829 non-null    float64
 16  포화지방산(g)   829 non-null    float64
 17  트랜스지방(g)   829 non-null    float64
dtypes: float64(10), int64(1), object(7)
memory usage: 116.7+ KB


In [32]:
kor_jap_lst = []
kor_jap_lst = data[data['한중일양'].isin(['한', '일'])]
ame_lst = data[data['한중일양'].isin(['양','중'])]
ame_df = pd.DataFrame(ame_lst)
ame_df

,FOOD_SEQ,음식명,레시피,한중일양,밥국반메면,중량,열량(kcal),탄수화물(g),단백질(g),지방(g)
41,42,고추잡채,https://www.youtube.com/embed/-CYp-sfT7gM,중,NaN,200.0,256.77,26.17,10.99,12.01
42,43,난자완스,https://www.youtube.com/embed/M_3fIYfiSMY,중,NaN,200.0,346.25,19.15,21.63,20.35
44,45,라조기,https://www.youtube.com/embed/JNhqP4USRUE,중,NaN,200.0,398.99,26.23,20.66,23.49
70,71,간자장,https://www.youtube.com/embed/_xw16_oGBFM,중,NaN,650.0,824.54,134.22,22.33,22.03
71,72,굴짬뽕,https://www.youtube.com/embed/fHT9gzhwzd4,중,NaN,900.0,680.67,115.69,32.87,9.61
...,...,...,...,...,...,...,...,...,...,...
692,7192,스크램블드에그,https://www.youtube.com/embed/nen6aasD_Sw,양,NaN,110.0,237.34,2.82,12.99,19.35
740,7240,양송이스프,https://www.youtube.com/embed/lDtKEhZHYmM,양,NaN,100.0,105.29,8.06,2.41,7.04
814,7314,깐풍기,https://www.youtube.com/embed/fGxgjJDlpS4,중,NaN,150.0,344.54,27.42,18.89,17.70
827,7327,탕수육,https://www.youtube.com/embed/mzZNNMM2WuQ,중,NaN,200.0,358.97,24.84,27.21,16.75


In [33]:
need_cal=user['cal'] /3
need_crb=user['crb'] /3
need_protein=user['protein'] /3
need_fat = user['fat'] /3
print(need_cal,need_crb,need_fat,need_protein)

878.3333333333334 109.79166666666667 34.157407407407405 32.9375


In [34]:
kor_df = pd.DataFrame(kor_jap_lst)
hi=kor_df[kor_df['밥국반메면']=='밥']
kor_df

,FOOD_SEQ,음식명,레시피,한중일양,밥국반메면,중량,열량(kcal),탄수화물(g),단백질(g),지방(g)
1,2,닭갈비,https://www.youtube.com/embed/HI8DlDs1gko,한,메,400.0,595.61,44.90,45.90,25.80
2,3,닭갈비,https://www.youtube.com/embed/HI8DlDs1gko,한,메,300.0,558.47,23.10,45.50,31.60
3,4,닭꼬치,https://www.youtube.com/embed/1xrcjy6eX7M,한,메,70.0,176.72,13.35,11.56,8.57
4,5,더덕구이,https://www.youtube.com/embed/ir4Z2IyFIvM,한,반,100.0,184.00,31.10,3.10,5.20
5,6,도미구이,https://www.youtube.com/embed/Q0W5ag8ZbzU,한,메,250.0,397.00,2.10,61.10,16.50
...,...,...,...,...,...,...,...,...,...,...
822,7322,새우튀김,https://www.youtube.com/embed/EPLbReYKHC8,한,메,150.0,430.34,28.44,25.52,23.84
823,7323,소시지튀김,https://www.youtube.com/embed/3zrCyzg0xdk,한,메,200.0,707.98,8.01,28.13,62.60
824,7324,오징어튀김,https://www.youtube.com/embed/K7KCOvEKPbg,한,메,150.0,505.71,33.65,18.56,32.99
825,7325,채소튀김,https://www.youtube.com/embed/1uvcqAUjH8A,한,메,100.0,404.63,36.83,3.25,27.14


In [37]:
data2

'FOOD_SEQ'

In [35]:
selected_lst=[]
for data in ame_df:
    if (need_cal*0.95) <= data2['열량(kcal)'] <= (need_cal*1.05):
        if (need_crb*0.95)<= data2['탄수화물(g)'] <= (need_crb*1.05):
            if (need_protein*0.95)<= data2['단백질(g)'] <=(need_protein*1.05):
                if (need_fat*0.95)<=data2['지방(g)'] <=(need_fat*1.05):
                    selected_lst.append(data)
                else:
                    print('성분에 맞는 음식이 없습니다')

TypeError: string indices must be integers

In [36]:
selected_lst = []
for index, row in data.iterrows():
    if (need_cal*0.95) <= row['열량(kcal)'] <= (need_cal*1.05):
        if (need_crb*0.95) <= row['탄수화물(g)'] <= (need_crb*1.05):
            if (need_protein*0.95) <= row['단백질(g)'] <= (need_protein*1.05):
                selected_lst.append(row.to_dict())
print(selected_lst)

AttributeError: 'str' object has no attribute 'iterrows'

In [ ]:
# x = 정수 입니다. 여기서 정수는 Food_seq-1 인 값입니다.
def eat_M(dic, df):
  # 여기서 dic은 사용자의 정보
    cal = dic['cal']
    crb = dic['crb']
    protein = dic['protein']
    fat = dic['fat']
        
    # 먹어야 할 칼로리
    need_cal = cal/3
    need_crb = crb/3
    need_protein = protein/3 
    need_fat = fat/3
    
    # 일단 담아둘 리스트
    selected_lst = []
    kor_jap_lst = []
    ame_chi_lst=[]
    
    kor_jap_lst = df[df['한중일양'].isin(['한', '일'])]
    ame_chi_lst = df[df['한중일양'].isin(['중', '양'])]
    
    # 음식 나라 타입이 한식 일식인 경우~
    kor_df = pd.DataFrame(kor_jap_lst)
    
    # 각각을 음식 타입에 따라 나누기
    rice_df=kor_df[kor_df['밥국반메면']=='밥']
    soup_df=kor_df[kor_df['밥국반메면']=='국']
    sub_df=kor_df[kor_df['밥국반메면']=='반']
    main_df=kor_df[kor_df['밥국반메면']=='메']
    noodle_df=kor_df[kor_df['밥국반메면']=='면']
    
    # 나라 타입이 중식과 양식일 경우~
    ame_df = pd.DataFrame(ame_chi_lst)
    
    for data in df:
        if data in kor_jap_lst:
            # 한, 일
            
    for data in ame_df:
        if (need_cal*0.95) <= data['열량(kcal)'] <= (need_cal*1.05):
            if (need_crb*0.95)<= data['탄수화물(g)'] <= (need_crb*1.05):
                if (need_protein*0.95)<= data['단백질(g)'] <=(need_protein*1.05):
                    if (need_fat*0.95)<=data['지방(g)'] <=(need_fat*1.05):
                        selected_lst.append(data)
                    else:
                        print('성분에 맞는 음식이 없습니다')

In [ ]:
# x = 정수 입니다. 여기서 정수는 Food_seq-1 인 값입니다.
def eat_L(dic, df, x):
  # 여기서 dic은 사용자의 정보
    cal = dic['cal']
    crb = dic['crb']
    protein = dic['protein']
    fat = dic['fat']
    

    


In [ ]:
# x = 정수 입니다. 여기서 정수는 Food_seq-1 인 값입니다.
def eat_D(dic, df, x):
  # 여기서 dic은 사용자의 정보
    cal = dic['cal']
    crb = dic['crb']
    protein = dic['protein']
    fat = dic['fat']
    
  # df는 데이터베이스에 있는 친구 (영양정보데이터)
    food_cal = df.loc[x,'열량(kcal)']
    food_crb = df.loc[x,'탄수화물(g)']
    food_protein = df.loc[x,'단백질(g)']
    food_fat = df.loc[x,'지방(g)']
    food_nation = df.loc[x,'한중일양']
    food_type = df.loc[x,'밥국반메면']

In [40]:
# x = 정수 입니다. 여기서 정수는 Food_seq-1 인 값입니다.
def recommend_gara(dic, df, x,code, now_code):
  # 여기서 dic은 사용자의 정보
    cal = dic['cal']
    crb = dic['crb']
    protein = dic['protein']
    fat = dic['fat']
    
  # df는 데이터베이스에 있는 친구 (영양정보데이터)
    food_cal = df.loc[x,'열량(kcal)']
    food_crb = df.loc[x,'탄수화물(g)']
    food_protein = df.loc[x,'단백질(g)']
    food_fat = df.loc[x,'지방(g)']

    # 아점저 코드를 받는다면? 
    # 아 : M  점 : L  저 : D
    # flask로 받는데 이름을 정해야 함. ==> 여기선 code로 갑시다.
    if 'M' in code and 'L' not in code and 'D' not in code:
      # 아침만 먹은 상황 점심을 먹을 상황
        cal = (cal - food_cal)/2
        crb = (crb - food_crb)/2
        protein = (protein - food_protein)/2
        fat = (fat - food_fat)/2
        

  
        return
    elif 'M' in code and 'L' in code and 'D' not in code:
      # 아침과 점심을 먹은 상황 저녁 먹을상황
  
      return
    elif 'M' not in code and 'L' not in code and 'D' not in code:
    # 아무것도 안먹었을 경우 => 아침을 먹어야 할 상황
  
      return
    elif 'M' not in code and 'L' in code and 'D' not in code:
      # 아침X 점심만 먹은 경우 저녁을 먹을 상황
        cal = (cal - food_cal)/2
        crb = (crb - food_crb)/2
        protein = (protein - food_protein)/2
        fat = (fat - food_fat)/2

        return
    else : 
      # 전혀 다른 경우
  
      return